In [60]:
import pandas
import re
import requests

In [35]:
samplePdfFile = "Camelyon17_.pdf"

In [36]:
convert_from_path(samplePdfFile)

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x121FA3470>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x121FA34E0>]

In [73]:
# USAGE
# python ocr.py --image images/example_01.png 
# python ocr.py --image images/example_02.png  --preprocess blur

# import the necessary packages
from PIL import Image
import pytesseract
import cv2
import os
import numpy

from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

def textFromPdfUrl(typeOfPreprocessing, pdfUrl) -> str:
        
    try:
        response = requests.get(pdfUrl)
    except requests.exceptions.RequestException as e:  
        raise SystemExit(e)
       
    # load the example image and convert it to grayscale
    pagesOfPdf = convert_from_bytes(response.content)
    text = ""

    for page in pagesOfPdf :

        numpy_image = numpy.array(page) 
        gray = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2GRAY)

        cv2.imshow("image", gray)

        # check to see if we should apply thresholding to preprocess the
        # image
        if typeOfPreprocessing == "thresh":
            gray = cv2.threshold(gray, 0, 255,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        # make a check to see if median blurring should be done to remove
        # noise
        elif typeOfPreprocessing == "blur":
            gray = cv2.medianBlur(gray, 3)

        # write the grayscale image to disk as a temporary file so we can
        # apply OCR to it
        filename = "{}.png".format(os.getpid())
        cv2.imwrite(filename, gray)

        # load the image as a PIL/Pillow image, apply OCR, and then delete
        # the temporary file
        text += pytesseract.image_to_string(Image.open(filename)).rstrip()
        os.remove(filename)

    noHyphenationText = text.replace("-\n", "\n")
    text = re.sub('(?<![\r\n])(\r?\n|\n?\r)(?![\r\n])', ' ', noHyphenationText)

    return text

In [76]:
samplePdfUrl = "https://grand-challenge-public.s3.amazonaws.com/evaluation-supplementary/80/46fc579c-51f0-40c4-bd1a-7c28e8033f33/Camelyon17_.pdf"

sampleText = textFromPdfUrl("thresh", samplePdfUrl)
print(sampleText)

SystemExit: HTTPSConnectionPool(host='grand-challenge-public.s3.amazonaws.cmom', port=443): Max retries exceeded with url: /evaluation-supplementary/80/46fc579c-51f0-40c4-bd1a-7c28e8033f33/Camelyon17_.pdf (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11e76ec50>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

/Users/theoevans/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [90]:
camelyonPapers = open("camelyon_papers", "r").read()
camelyonPapersUrls = camelyonPapers.split()
numberOfPapers = len(camelyonPapersUrls)

paperIdRegex = "(?<=80\/).*(?=\/)"
preprocessing = "thresh"
documentData = []

for idx, url in enumerate(camelyonPapersUrls) :
    paperId = re.search(paperIdRegex, url).group()[-12:]
    paperLanguage = "en"
    paperText = textFromPdfUrl(preprocessing, url)
    
    if len(paperText) > 5120 :
        
    
    document = {"id" : paperId,
                "language" : paperLanguage, 
                "text" : paperText}
    
    print(paperId + ": " + str(idx) + " of " + str(numberOfPapers) + " complete")
    documentData.append(document)
    
documents = {"documents" : documentData}

7c28e8033f33: 0 of 49 complete
e1b12cd64c7a: 1 of 49 complete
215b635b77ff: 2 of 49 complete
32a1c33dc208: 3 of 49 complete
38b78f82d298: 4 of 49 complete
321130b416dd: 5 of 49 complete
2b070c9bfbd4: 6 of 49 complete
7a1c54ae5035: 7 of 49 complete
548ba1cfcb36: 8 of 49 complete
ed96a7556d66: 9 of 49 complete
fe69a1e957c1: 10 of 49 complete
a98658870e94: 11 of 49 complete
1b44c2ede325: 12 of 49 complete
af0b585dc479: 13 of 49 complete
4e4fd9627a02: 14 of 49 complete
ba1fb0498798: 15 of 49 complete
89327b3a76ee: 16 of 49 complete
943f8075e76a: 17 of 49 complete
a7805f980bab: 18 of 49 complete
1f53a98be05e: 19 of 49 complete
16b60ef7854f: 20 of 49 complete
a3f9a175c25f: 21 of 49 complete
f1f2509854c7: 22 of 49 complete
72de07f455df: 23 of 49 complete
d93cb1ac766e: 24 of 49 complete
8eb10fb9821b: 25 of 49 complete
dadc927339f4: 26 of 49 complete
f8d755a079fa: 27 of 49 complete
7440702059c7: 28 of 49 complete
bf31c7967cdd: 29 of 49 complete
454ef427cf01: 30 of 49 complete
a29a0501bbb5: 31 o

In [92]:
import requests
# pprint is used to format the JSON response
from pprint import pprint

subscription_key = "3cd5618e7a434c5eace1d0ca61762a66"
endpoint = "https://westcentralus.api.cognitive.microsoft.com"

keyphrase_url = endpoint + "/text/analytics/v2.1/keyphrases"

headers = {"Ocp-Apim-Subscription-Key": subscription_key}
response = requests.post(keyphrase_url, headers=headers, json=documents)
key_phrases = response.json()
pprint(key_phrases)

{'documents': [{'id': '32a1c33dc208',
                'keyPhrases': ['tuned hyperparameters',
                               'tree parzen estimator',
                               'time',
                               'random forest']},
               {'id': '38b78f82d298',
                'keyPhrases': ['University of Manchester',
                               'Kapodistrian University of Athens',
                               'Manchester Cancer Research Centre',
                               'Academy of Athens',
                               'Manchester University NHS Foundation Trust',
                               'UK',
                               'Greece',
                               'NIHR Manchester Biomedical Research Centre',
                               'School of Medicine',
                               'Manchester Academic Health Science Centre',
                               'Biomedical Research Foundation',
                               'Precision Medicine